# Don't work...

In [41]:
import os
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


root = "./data"

transform_train = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5),(0.5))]
)

transform_val = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5),(0.5))]
)

f_mnist_train = datasets.CIFAR10(
    root=root,
    download=True,
    train=True,
    transform=transform_train,
)
f_mnist_test = datasets.CIFAR10(
    root=root,
    download=True,
    train=False,
    transform=transform_val
)

batch_size = 64
train_dataloader = DataLoader(f_mnist_train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(f_mnist_test, batch_size=batch_size, shuffle=False)

for (x,t) in train_dataloader:
    print(x.shape)
    print(t.shape)
    break

for (x,t) in test_dataloader:
    print(x.shape)
    print(t.shape)
    break


Files already downloaded and verified
Files already downloaded and verified
torch.Size([64, 3, 32, 32])
torch.Size([64])
torch.Size([64, 3, 32, 32])
torch.Size([64])


In [42]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        """(batch_size, 3, 32, 32) -> (batch_size,  32, 32)"""
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            padding=True,
            padding_mode="zeros"
            )

        self.conv2 = nn.Conv2d(
            in_channels=32,
            out_channels=32,
            kernel_size=3, 
            padding=True, 
            padding_mode="zeros")

        self.pool1 = nn.MaxPool2d(2, 2)

        self.dropout1 = nn.Dropout2d(0.5)

        self.conv3 = nn.Conv2d(
            in_channels=32, 
            out_channels=64, 
            kernel_size=3, 
            padding=True, 
            padding_mode="zeros")

        self.conv4 = nn.Conv2d(
            in_channels=64, 
            out_channels=64, 
            kernel_size=3, 
            padding=True, 
            padding_mode="zeros")

        self.pool2 = nn.MaxPool2d(2,2)

        self.dropout2 = nn.Dropout2d(0.5)

        self.fc1 = nn.Linear(64*8*8, 512)
        
        self.dropout3 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(F.relu(self.conv2(x)))
        x = self.dropout1(x)
        x = F.relu(self.conv3(x))
        x = self.pool2(F.relu(self.conv4(x)))
        x = self.dropout2(x)
        x = x.view(-1, 64*8*8) #Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)
        return x
        



In [43]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device=device)
model
print(device)

cpu


In [44]:
import torch.optim

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [45]:
def train_step(x, t):
    model.train()
    preds = model(x)
    loss = criterion(preds, t)
    optimizer.zero_grad()
    loss.backword() #process of backword
    optimizer.step()
    return loss, preds

In [46]:
def test_step(x, t):
    model.eval()
    preds = model(x)
    loss = criterion(preds, t)
    return loss, preds

In [47]:
%%time
'''
7.モデルを使用して学習する
'''
from sklearn.metrics import accuracy_score

# エポック数
epochs = 120
# 損失と精度の履歴を保存するためのdictオブジェクト
history = {'loss':[],'accuracy':[], 'test_loss':[], 'test_accuracy':[]}

# 収束が停滞したら学習率を減衰するスケジューラー
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,   # オプティマイザーを指定
    mode='max',  # 監視対象は最大値
    factor=0.5,  # 学習率を減衰する割合
    patience=10, # 監視対象のエポック数
    min_lr=0.0001, # 最小学習率
    verbose=True # 学習率を減衰した場合に通知する
    )

# 学習を行う
for epoch in range(epochs):
    train_loss = 0. # 訓練1エポックあたりの損失を保持する変数
    train_acc = 0.  # 訓練1エポックごとの精度を保持する変数
    test_loss = 0.  # 評価1エポックごとの損失を保持する変数
    test_acc = 0.   # 評価1エポックごとの精度を保持する変数

    # 1ステップにおける訓練用ミニバッチを使用した学習
    for (x, t) in train_dataloader:
        # torch.Tensorオブジェクトにデバイスを割り当てる
        x, t = x.to(device), t.to(device)
        loss, preds = train_step(x, t) # 損失と予測値を取得
        train_loss += loss.item()      # ステップごとの損失を加算
        train_acc += accuracy_score(
            t.tolist(),
            preds.argmax(dim=-1).tolist()
        )                              # ステップごとの精度を加算

    # 1ステップにおけるテストデータのミニバッチを使用した評価
    for (x, t) in test_dataloader:
        # torch.Tensorオブジェクトにデバイスを割り当てる
        x, t = x.to(device), t.to(device)
        loss, preds = test_step(x, t) # 損失と予測値を取得
        test_loss += loss.item()       # ステップごとの損失を加算
        test_acc += accuracy_score(
            t.tolist(),
            preds.argmax(dim=-1).tolist()
        )                              # ステップごとの精度を加算

    # 訓練時の損失の平均値を取得
    avg_train_loss = train_loss / len(train_dataloader)
    # 訓練時の精度の平均値を取得
    avg_train_acc = train_acc / len(train_dataloader)
    # 検証時の損失の平均値を取得
    avg_test_loss = test_loss / len(test_dataloader)
    # 検証時の精度の平均値を取得
    avg_test_acc = test_acc / len(test_dataloader)

    # 訓練データの履歴を保存する
    history['loss'].append(avg_train_loss)
    history['accuracy'].append(avg_train_acc)
    # テストデータの履歴を保存する
    history['test_loss'].append(avg_test_loss)
    history['test_accuracy'].append(avg_test_acc)

    # 1エポックごとに結果を出力
    if (epoch + 1) % 1 == 0:
        print(
            'epoch({}) train_loss: {:.4} train_acc: {:.4} val_loss: {:.4} val_acc: {:.4}'.format(
                epoch+1,
                avg_train_loss, # 訓練データの損失を出力
                avg_train_acc,  # 訓練データの精度を出力
                avg_test_loss,  # テストデータの損失を出力
                avg_test_acc    # テストデータの精度を出力
    ))
    # スケジューラー、テストデータの精度を監視する
    scheduler.step(avg_test_acc)

TypeError: conv2d() received an invalid combination of arguments - got (Tensor, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (Tensor, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (Tensor, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
